In [1]:
# import logging
# logging.basicConfig(level=logging.ERROR)

In [2]:
from langchain.llms import LlamaCpp, GPT4All
from langchain.embeddings import LlamaCppEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever

In [ ]:
# local_path = "/Users/jon/Downloads/ggml-vicuna-13b-1.1-q4_2.bin"
# llm = LlamaCpp(model_path=local_path, verbose=True, use_mmap=False, n_ctx=2048)

local_path = "/Users/jon/Downloads/gpt4all-lora-unfiltered-quantized.bin"
llm = GPT4All(model=local_path, n_ctx=2048)

In [ ]:
USER_NAME = "Jon" # The name you want to use when interviewing the agent.

In [ ]:
from retrivers.llama_time_weighted_retriever import LlamaTimeWeightedVectorStoreRetriever
from vectorestores.chroma import EnhancedChroma

def create_new_memory_retriever():
    embeddings_model = LlamaCppEmbeddings(model_path=local_path)
    vs = EnhancedChroma(embedding_function=embeddings_model)
    return LlamaTimeWeightedVectorStoreRetriever(vectorstore=vs, other_score_keys=["importance"], k=15)

In [ ]:
from generative_agents.llama_generative_agent import LlamaGenerativeAgent
from generative_agents.llama_memory import LlamaGenerativeAgentMemory

tommies_memory = LlamaGenerativeAgentMemory(
    llm=llm,
    memory_retriever=create_new_memory_retriever(),
    reflection_threshold=8, # we will give this a relatively low number to show how reflection works
    verbose=True,
)

tommie = LlamaGenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative", # You can add more persistent traits here
    status="looking for a job", # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=llm,
    memory=tommies_memory,
    verbose=True,
)

In [ ]:
# print(tommie.get_summary())

In [ ]:
# We can add memories directly to the memory object
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [ ]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

In [ ]:
def interview_agent(agent: LlamaGenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [ ]:
interview_agent(tommie, "What are you most worried about today?")


In [ ]:
# interview_agent(tommie, "What are you looking forward to doing today?")


In [ ]:
# interview_agent(tommie, "What are you most worried about today?")


In [ ]:
# interview_agent(tommie, "Are you in love?")